<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

#  Summary
This notebook looks through the reddit whisky reviews and searches on vintages.com for the whisky name and returns any results. The results of the search are recorded into parquet files depending on whether they have no match, one match ,or multiple matches.

At the moment all I am looking for is the product ID so we can pull product details from the lcbo api in the next step.

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib.parse
import urllib.request
import re
import time
from urllib.error import HTTPError

In [3]:
rdb = pd.read_excel('data/redditwhiskeydatabase.xlsx')

In [4]:
rdb.head()

,Timestamp,Whisky Name,Reviewer's Reddit Username,Link To Reddit Review,Reviewer Rating,Whisky Region or Style,Full Bottle Price Paid,Date of Review
0,2012-12-14 10:03:18,100 Pipers,merlinblack,http://www.reddit.com/r/Scotch/comments/14uder...,68,Blend,NaN,12/14/12
1,2018-11-04 00:50:38.789000,1792 225th Anniversary,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ssq...,8,Bourbon,NaN,10/30/18
2,2017-11-17 08:15:21.922000,1792 225th Anniversary,WildOscar66,https://www.reddit.com/r/bourbon/comments/7445...,80,Bourbon,NaN,10/17/17
3,2018-11-04 00:49:14.921000,1792 Bottled In Bond,scottmotorrad,https://www.reddit.com/r/bourbon/comments/9ryi...,80,Bourbon,34.99,10/27/18
4,2018-12-19 00:38:02.415000,1792 Bottled In Bond Oak Liquor Cabinet Pick,scottmotorrad,https://www.reddit.com/r/bourbon/comments/a7jw...,90,Bourbon,NaN,12/19/2018


In [5]:
uniquewhiskies = rdb.assign(Whisky_Name = lambda x: x['Whisky Name'].str.strip())['Whisky_Name'].unique()

In [6]:
print(rdb.shape)
print(uniquewhiskies.shape)

(31892, 8)
(10151,)


In [7]:
uniquewhiskies = pd.DataFrame(uniquewhiskies, columns=['RedditWhiskyName']).reset_index().rename({'index':'RedditWhiskyID'},axis='columns')

In [ ]:
def productSearch(itemname):
    # strip out spaces and non ascii characters
    itemname = re.sub(r'[^\x00-\x7F]+',' ',  itemname.replace(' ','%20'))
    url = 'http://www.vintages.com/lcbo-ear/vintages/product/searchResults.do?ITEM_NAME=' + itemname.replace(' ','%20')
    user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
    values = {'name' : 'Firstname Lastname',
              'location' : 'Toronto',
              'language' : 'Python' }
    headers = { 'User-Agent' : user_agent }
    print(url)
    data = urllib.parse.urlencode(values)
    data = data.encode('ascii')
    req = urllib.request.Request(url, data, headers)
    try:
        with urllib.request.urlopen(req) as response:
           content = response.read()
        return BeautifulSoup(content, 'html.parser')
    except :
        return None

In [ ]:
def getProductIds(itemname):
    result = productSearch(itemname)
    if result is None:
        return None
    matches = pd.DataFrame(columns=['Name','ProductID'])
    if bool(re.search('Search',str(result.title))):
        # More than one result
        tables = result.findAll("table", {"class": "search"})    
        for table in tables:
            row = table.findAll('a',href=True)
            if len(row) == 0:
                return None
            else:
                link = row[1]['href']
                name = row[1].text.strip()
                productid = re.search('(?<=itemNumber\=)[0-9]*', link)[0]
                matches = matches.append(pd.DataFrame([[name,productid]], columns=['Name','ProductID']),ignore_index=True)
    else: 
        # Only one result
        name = result.find("input",{"name":"itemName"})['value']
        productid = result.find("input",{"name":"itemNumber"})['value']
        matches = matches.append(pd.DataFrame([[name,productid]], columns=['Name','ProductID']),ignore_index=True)
    
    return matches

In [8]:
uniquewhiskies.iloc[2500]

RedditWhiskyID                           2500
RedditWhiskyName    Caol Ila 12 Cask strength
Name: 2500, dtype: object

In [ ]:
nomatch = pd.DataFrame(columns=['RedditWhiskyID','RedditWhiskyName'])
onematch = pd.DataFrame(columns=['RedditWhiskyID','RedditWhiskyName','Name','ProductID'])
product_ids = pd.DataFrame(columns=['RedditWhiskyID','RedditWhiskyName','Name','ProductID'])
for whisky in uniquewhiskies.itertuples():
    myId = getProductIds(whisky.RedditWhiskyName)
    time.sleep(3)
    if myId is None:
        nomatch = nomatch.append(pd.DataFrame([[whisky.RedditWhiskyID,whisky.RedditWhiskyName]],
                                              columns=['RedditWhiskyID','RedditWhiskyName']))
    elif len(myId) == 0:
        nomatch = nomatch.append(pd.DataFrame([[whisky.RedditWhiskyID,whisky.RedditWhiskyName]],
                                              columns=['RedditWhiskyID','RedditWhiskyName']))
    elif len(myId) == 1:
        myId['RedditWhiskyID'] = whisky.RedditWhiskyID
        myId['RedditWhiskyName'] = whisky.RedditWhiskyName
        onematch = onematch.append(myId)
    else:
        myId['RedditWhiskyID'] = whisky.RedditWhiskyID
        myId['RedditWhiskyName'] = whisky.RedditWhiskyName
        product_ids = product_ids.append(myId)

continuing where we left off:


In [11]:
nomatch.to_parquet('nomatch.parquet')
onematch.to_parquet('onematch.parquet')
product_ids.to_parquet('product_ids.parquet')
uniquewhiskies.to_parquet('whisky_ids.parquet')

/opt/tljh/user/lib/python3.6/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)


In [ ]:
nomatch.shape

In [ ]:
onematch.shape

In [ ]:
product_ids.shape

In [9]:
uniquewhiskies

,RedditWhiskyID,RedditWhiskyName
0,0,100 Pipers
1,1,1792 225th Anniversary
2,2,1792 Bottled In Bond
3,3,1792 Bottled In Bond Oak Liquor Cabinet Pick
4,4,1792 Full Proof
5,5,1792 Full Proof Angel's Beverage
6,6,1792 Full Proof Binny's
7,7,1792 Full Proof Loch & K(e)y Microbatch
8,8,1792 Full Proof Lueken's Store Pick
9,9,1792 Full Proof Oak Liquor Cabinet Pick


In [ ]:
product_ids